In [2]:
import pandas as pd

In [ ]:
import pandas as pd

def agregar_columna_fecha(df):
    """Convierte columnas 'Año' y 'Mes' en una columna Period 'Fecha' (formato año-mes), colocándola como la primera columna."""
    meses = {
        'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4,
        'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8,
        'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
    }
    df = df.copy()
    df['Mes_num'] = df['Mes'].map(meses)
    df['Fecha'] = pd.to_datetime(dict(year=df['Año'], month=df['Mes_num'], day=1)).dt.to_period('M')
    df.drop(columns=['Mes_num', 'Año', 'Mes'], inplace=True)

    # Reordenar columnas para que 'Fecha' sea la primera
    cols = ['Fecha'] + [col for col in df.columns if col != 'Fecha']
    df = df[cols]

    return df


In [49]:
consumo = agregar_columna_fecha(pd.read_csv('limpias/consumo_laminados.csv'))
expo = agregar_columna_fecha(pd.read_csv('limpias/exportaciones_laminados.csv'))
impo = agregar_columna_fecha(pd.read_csv('limpias/importaciones_laminados.csv'))
prod = agregar_columna_fecha(pd.read_csv('limpias/produccion.csv')).drop(columns=['Material'])

In [50]:
dfs = [consumo, expo, impo, prod]

for df in dfs:
    print(df.shape)

(8976, 4)
(8448, 4)
(8448, 4)
(17856, 4)


In [20]:
consumo.head()

,Fecha,País,Producto,Consumo
0,2024-01,Argentina,Largos,114.84400
1,2024-01,Argentina,Planos,158.63700
2,2024-01,Argentina,Tubos Sin Costura,29.20700
3,2024-01,Argentina,Laminados,302.68800
4,2024-01,Bolivia,Largos,19.50208


In [21]:
expo.head()

,Fecha,País,Producto,Exportacion
0,2024-01,Argentina,Largos,7.790000
1,2024-01,Argentina,Planos,29.406000
2,2024-01,Argentina,Tubos Sin Costura,16.536000
3,2024-01,Argentina,Laminados,53.732000
4,2024-01,Bolivia,Largos,0.000192


In [22]:
impo.head()

,Fecha,País,Producto,Importacion
0,2024-01,Argentina,Largos,9.491000
1,2024-01,Argentina,Planos,39.365000
2,2024-01,Argentina,Tubos Sin Costura,3.343000
3,2024-01,Argentina,Laminados,52.199000
4,2024-01,Bolivia,Largos,19.502272


In [51]:
prod.head()

,Fecha,País,Producto,Producción
0,2024-01,Argentina,H.Eléctrico,240.998
1,2024-01,Argentina,BOF,166.765
2,2024-01,Brasil,H.Eléctrico,656.539
3,2024-01,Brasil,BOF,2066.456
4,2024-01,Chile,H.Eléctrico,35.640


In [88]:
def transformar_y_unir(consumo, expo, impo, prod):
    # Función interna para pivotear y renombrar columnas
    def pivotar(df, valor, prefijo):
        df = df.copy()
        # Normalizar nombres
        df['col_nombre'] = (
            df['Producto'].str.lower()
            + '_'
            + df['País'].str.lower().str.replace(' ', '_')
        )
        df = df.pivot(index='Fecha', columns='col_nombre', values=valor)
        df.columns = [f"{prefijo}_{col}" for col in df.columns]
        return df

    # Pivotear cada tabla
    consumo_pivot = pivotar(consumo, 'Consumo', 'consumo')
    expo_pivot = pivotar(expo, 'Exportacion', 'expo')
    impo_pivot = pivotar(impo, 'Importacion', 'impo')
    prod_pivot = pivotar(prod, 'Producción', 'prod')

    # Unir por Fecha
    df_final = consumo_pivot.join([expo_pivot, impo_pivot, prod_pivot], how='inner')

    # Ordenar por fecha
    df_final.sort_index(inplace=True)

    return df_final


In [89]:
df_mes = transformar_y_unir(consumo, expo, impo, prod)


In [83]:
df_mes.head()

,consumo_laminados_argentina,consumo_laminados_bolivia,consumo_laminados_brasil,consumo_laminados_chile,consumo_laminados_colombia,consumo_laminados_costa_rica,consumo_laminados_ecuador,consumo_laminados_el_salvador,consumo_laminados_guatemala,consumo_laminados_honduras,...,prod_tubos sin costura_ecuador,prod_tubos sin costura_el_salvador,prod_tubos sin costura_guatemala,prod_tubos sin costura_méxico,prod_tubos sin costura_paraguay,prod_tubos sin costura_perú,prod_tubos sin costura_república_dominicana,prod_tubos sin costura_trinidad_y_tobago,prod_tubos sin costura_uruguay,prod_tubos sin costura_venezuela
Fecha,,,,,,,,,,,,,,,,,,,,,
2014-01,414.128173,50.434851,2008.7977,245.040834,325.800273,61.791907,169.416774,29.335623,81.662490,7.715310,...,0.0,0.0,0.0,91.125073,0.0,0.0,0.0,0.0,0.0,0.0
2014-02,389.813458,43.231787,2024.6590,138.316052,295.411541,53.746617,89.476312,28.823996,56.280563,19.135921,...,0.0,0.0,0.0,81.403017,0.0,0.0,0.0,0.0,0.0,0.0
2014-03,470.319287,40.837359,2398.5319,212.329678,317.983510,83.846634,153.185823,39.430051,77.288694,17.349350,...,0.0,0.0,0.0,88.883066,0.0,0.0,0.0,0.0,0.0,0.0
2014-04,472.415271,32.998635,2356.3058,242.738627,392.794638,71.292876,143.408089,18.553230,75.382410,7.260462,...,0.0,0.0,0.0,87.508544,0.0,0.0,0.0,0.0,0.0,0.0
2014-05,436.218640,44.592685,2333.1539,231.516427,355.319493,67.584168,135.982484,22.998549,83.447093,19.264620,...,0.0,0.0,0.0,91.046758,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
for column in df_mes.columns:
    print(column)

consumo_laminados_argentina
consumo_laminados_bolivia
consumo_laminados_brasil
consumo_laminados_chile
consumo_laminados_colombia
consumo_laminados_costa_rica
consumo_laminados_ecuador
consumo_laminados_el_salvador
consumo_laminados_guatemala
consumo_laminados_honduras
consumo_laminados_méxico
consumo_laminados_panamá
consumo_laminados_paraguay
consumo_laminados_perú
consumo_laminados_república_dominicana
consumo_laminados_uruguay
consumo_laminados_venezuela
consumo_largos_argentina
consumo_largos_bolivia
consumo_largos_brasil
consumo_largos_chile
consumo_largos_colombia
consumo_largos_costa_rica
consumo_largos_ecuador
consumo_largos_el_salvador
consumo_largos_guatemala
consumo_largos_honduras
consumo_largos_méxico
consumo_largos_panamá
consumo_largos_paraguay
consumo_largos_perú
consumo_largos_república_dominicana
consumo_largos_uruguay
consumo_largos_venezuela
consumo_planos_argentina
consumo_planos_bolivia
consumo_planos_brasil
consumo_planos_chile
consumo_planos_colombia
consumo_pl